In [22]:
import os

import numpy as np
import pandas as pd

import joblib

from typing import Dict

import yaml
import json

import warnings
warnings.filterwarnings("ignore")

In [23]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config['preprocessing']
training = config['train']
evaluate = config['evaluate']


# check columns with train
column_sequence_path = preproc['unique_values_path']
with open(column_sequence_path) as json_file:
    column_sequence = json.load(json_file)

In [24]:
data_test = pd.read_csv(evaluate['predict_path'])
data_test

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,10.37,0.10,N,4
39094,97739,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,6.03,0.15,N,3
39095,97740,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,7.51,0.25,N,25
39096,97741,22,36000,MORTGAGE,4.0,PERSONAL,D,14000,15.62,0.39,Y,4


# Preprocessing

In [31]:
def save_unique_train_data(
    data: pd.DataFrame, drop_columns: list, target_column: str, unique_values_path: str
) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param drop_columns: список с признаками для удаления
    :param data: датасет
    :param target_column: целевая переменная
    :param unique_values_path: путь до файла со словарем
    :return: None
    """
    unique_df = data.drop(
        columns=drop_columns, axis=1, errors="ignore"
    )
    # создаем словарь с уникальными значениями для вывода в UI
    dict_unique = {key: unique_df[key].unique().tolist() for key in unique_df.columns}
    with open(unique_values_path, "w") as file:
        json.dump(dict_unique, file)
        
        
def check_columns_evaluate(data: pd.DataFrame, unique_values_path: str) -> pd.DataFrame:
    """
    Проверка на наличие признаков из train и упорядочивание признаков согласно train
    :param data: датасет test
    :param unique_values_path: путь до списка с признаками train для сравнения
    :return: датасет test
    """
    with open(unique_values_path) as json_file:
        unique_values = json.load(json_file)

    column_sequence = unique_values.keys()

    assert set(column_sequence) == set(data.columns), "Разные признаки"
    return data[column_sequence]


        
def replace_values(data: pd.DataFrame, map_change_columns: dict) -> pd.DataFrame:
    """
    Замена значений в датасете
    :param data: датасет
    :param map_change_columns: словарь с признаками и значениями
    :return: датасет
    """
    return data.replace(map_change_columns)

def transform_types(data: pd.DataFrame, change_type_columns: dict) -> pd.DataFrame:
    """
    Преобразование признаков в заданный тип данных
    :param data: датасет
    :param change_type_columns: словарь с признаками и типами данных
    :return:
    """
    return data.astype(change_type_columns, errors="raise")

def filling_nan(data: pd.DataFrame) -> pd.DataFrame:
    """
    Заполнение пропущенных значений в датасете
    :param data: датасет
    :return: датасет с заполненнными пропусками
    """
    # Создаем список из колонок с пропущенными значениями
    nan_col_list = data.isna().sum().to_frame(name="nans").query("nans > 0").index
    
    # Заполняем пропуски в колонках: столбцы с числовыми данными заполняем медианой, с категориальными модой
    for col in nan_col_list:
        if data[col].dtype == int or data[col].dtype == float:
            data[col].fillna(data[col].median(), inplace=True)
        else:
            data[col].fillna(data[col].mode()[0], inplace=True)
    
    return data

def get_bins(data: (int, float), first_val: (int, float),
             second_val: (int, float)) -> str:
    """
    Генерация столбцов с бинами для различных числовых признаков
    :param data: датасет
    :param first_val: первый порог значения для разбиения на бины
    :param second_val: второй порог значения для разбиения на бины
    :return: датасет
    """
    assert isinstance(data, (int, float)), "Проблема с типом данных в признаке"
    result = ("small" if data <= first_val else
              "medium" if first_val < data <= second_val else "large")
    return result

In [32]:
def pipeline_preprocess(data: pd.DataFrame, flg_evaluate: bool = True, **kwargs):
    """
    Пайплайн по предобработке данных
    :param data: датасет
    :param flg_evaluate: флаг для evaluate
    :return: датасет
    """
    
    # проверка на совпадение с признаками из train, либо сохранение уникальных данных с признаками из train
    if flg_evaluate:
        data = check_columns_evaluate(
            data=data, unique_values_path=kwargs["unique_values_path"]
        )
    else:
        save_unique_train_data(
            data=data,
            drop_columns=kwargs["drop_columns"],
            target_column=kwargs["target_column"],
            unique_values_path=kwargs["unique_values_path"],
        )
    
    # удаление колонок
    data = data.drop(kwargs["drop_columns"], axis=1, errors="ignore")

    # замена значений
    data = replace_values(data=data, map_change_columns=kwargs["map_change_columns"])
    
    # заполнение пропусков
    data = filling_nan(data)
    
    # bins
    assert isinstance(kwargs["map_bins_columns"], dict), "Подайте тип данных для бинаризации в формате dict"
    
    for key in kwargs["map_bins_columns"].keys():
        data[f"{key}_bins"] = data[key].apply(
            lambda x: get_bins(
                x,
                first_val=kwargs["map_bins_columns"][key][0],
                second_val=kwargs["map_bins_columns"][key][1],
            )
        )
        
    # изменение столбцов типа object на category
    dict_category = {key: "category" for key in data.select_dtypes(["object"]).columns}
    data = transform_types(data=data, change_type_columns=dict_category)
    return data

In [33]:
data_pred_test = pipeline_preprocess(data=data_test, **preproc)

In [34]:
data_pred_test.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_age_bins,person_income_bins,person_emp_length_bins,loan_amnt_bins,loan_int_rate_bins,loan_percent_income_bins,cb_person_cred_hist_length_bins
0,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,0,2,small,medium,small,large,large,large,small
1,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,1,4,small,large,medium,medium,medium,small,small
2,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,1,2,small,small,medium,small,large,medium,small
3,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,0,7,small,medium,medium,small,medium,medium,medium
4,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,1,4,small,large,medium,large,large,medium,small


In [35]:
data_pred_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   person_age                       39098 non-null  int64   
 1   person_income                    39098 non-null  int64   
 2   person_home_ownership            39098 non-null  category
 3   person_emp_length                39098 non-null  float64 
 4   loan_intent                      39098 non-null  category
 5   loan_grade                       39098 non-null  category
 6   loan_amnt                        39098 non-null  int64   
 7   loan_int_rate                    39098 non-null  float64 
 8   loan_percent_income              39098 non-null  float64 
 9   cb_person_default_on_file        39098 non-null  int64   
 10  cb_person_cred_hist_length       39098 non-null  int64   
 11  person_age_bins                  39098 non-null  category
 12  pers

# Evaluate

In [36]:
model = joblib.load(training['model_path'])

In [37]:
data_pred_test['predict'] = model.predict(data_pred_test)

In [38]:
data_pred_test

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_age_bins,person_income_bins,person_emp_length_bins,loan_amnt_bins,loan_int_rate_bins,loan_percent_income_bins,cb_person_cred_hist_length_bins,predict
0,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,0,2,small,medium,small,large,large,large,small,1
1,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,1,4,small,large,medium,medium,medium,small,small,0
2,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,1,2,small,small,medium,small,large,medium,small,0
3,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,0,7,small,medium,medium,small,medium,medium,medium,0
4,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,1,4,small,large,medium,large,large,medium,small,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,10.37,0.10,0,4,small,small,small,small,medium,small,small,0
39094,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,6.03,0.15,0,3,small,medium,medium,small,small,medium,small,0
39095,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,7.51,0.25,0,25,medium,medium,small,large,small,medium,large,0
39096,22,36000,MORTGAGE,4.0,PERSONAL,D,14000,15.62,0.39,1,4,small,small,medium,large,large,large,small,0
